In [1]:
import numpy as np
from grapy import np_extensions as ne
from grapy import grapy as gp

In [2]:
#a = np.array([[1,2],[2,3],[3,4]])
#b = np.array([[1,2],[2,4]])
a = np.array([[1,2],[2,3]])
b = np.array([[1,2]])

np.expand_dims(b, 1) == a
#np_extensions.contains_vec(a,b, axis=0)

array([[[ True,  True],
        [False, False]]])

In [3]:
np.array([1, 2]).shape == (2, 2)

False

In [4]:
#g = Graph.from_edges(np.array([[1,2], [2,3], [3,1], [3,4], [4,5], [5,6], [6,4], [6,1]]))
g = gp.from_edges([[1,2], [2,3], [3,1], [3,4], [4,5], [5,6], [6,4], [6,1]])
print(g)

([1 2 3 4 5 6]
[[1 2]
 [2 3]
 [3 1]
 [3 4]
 [4 5]
 [5 6]
 [6 4]
 [6 1]])


In [5]:
(
    g.verts,
    g.edges,
    len(g),
    g.get_edges([1, 2, 3], [4, 5, 6]),
    g.neighb([1, 3]),
    1 in g,
    7 in g,
    [1,2] in g,
    [1,5,7] in g,
    [[1,2],[2,3]] in g,
    [[1,2],[2,5]] in g,
    gp.from_edges([[1,2],[2,3]]) in g,
    gp.from_edges([[1,2],[2,5]]) in g,
    g[[1,2,3,5]],
)

(array([1, 2, 3, 4, 5, 6]),
 array([[1, 2],
        [2, 3],
        [3, 1],
        [3, 4],
        [4, 5],
        [5, 6],
        [6, 4],
        [6, 1]]),
 6,
 array([[3, 4],
        [6, 1]]),
 array([1, 2, 3, 4, 6]),
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 ggraph(
 edges=array([1, 2, 3, 5]),
 verts=array([[1, 2],
        [2, 3],
        [3, 1]])))

In [6]:
a = np.array([1,2,3])
b = np.array([1,2,4])

a[(np.expand_dims(b, 1) != a).all(axis=0)], np.setdiff1d(a,b), np.union1d(a,b)

(array([3]), array([3]), array([1, 2, 3, 4]))

In [7]:
a = np.array([[1,2], [3,4]])
b = np.array([1,2])

np.setdiff1d(a,b)

array([3, 4])

In [17]:
A = np.array([[1,2], [2,3], [3,1]])
B = np.array([1,2])

dims = np.maximum(B.max(0),A.max(0))+1
out = A[~np.in1d(np.ravel_multi_index(A.T,dims),np.ravel_multi_index(B.T,dims))]

out

TypeError: only integer scalar arrays can be converted to a scalar index

In [9]:
~3

-4

In [11]:
g = gp.from_edges([[1,2], [2,3], [3,4], [4,5]])

g-[[2,3], [3,4]], g-[[2,3], [4,3]]

(ggraph(
 edges=array([1, 2, 3, 4, 5]),
 verts=array([1, 5])),
 ggraph(
 edges=array([1, 2, 3, 4, 5]),
 verts=array([1, 5])))

In [16]:
g = gp.from_edges([[1,2], [2,3], [3,4], [4,5]])
edges = [[2,3], [3,4]]

remaining_edges = np.setdiff1d(g.edges, edges)
print(remaining_edges)
remaining_edges = np.setdiff1d(remaining_edges, np.flip(edges, axis=1))
remaining_edges

[1 5]


array([1, 5])

# ideas
- sort edges for easy compability
- methods to add:
  - remove single verts